<a href="https://colab.research.google.com/github/Muromoto/michaelshannon/blob/master/Buy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install twilio
# from twilio.rest import Client

from google.colab import auth # Remove when migrated
auth.authenticate_user() # Remove when migrated

from bs4 import BeautifulSoup
import requests
import lxml
import string
import pandas as pd
import time
import gspread
import json
import numpy as np
from datetime import datetime as dt, date, timedelta
from fractions import Fraction

from google.auth import default # Remove when migrated
creds, _ = default() # Remove when migrated
gc = gspread.authorize(creds) # Remove when migrated

In [ ]:
# Pull in important Twilio creds from Excel sheet
# twil_data = pd.read_excel('/content/drive/MyDrive/Python Projects/twil_data.xlsx')

# Set Twilio variables needed to send a message
# account_sid = twil_data.iloc[0]['account_sid']
# auth_token = twil_data.iloc[0]['auth_token']
# client = Client(account_sid, auth_token)

# Create an error indicator
# error = False

# Send an SMS to indicate not to change the sheet
# try :
#   message = client.messages.create(
#     body="The video game script is running. Do not make any changes to the spreadsheet.",
#     from_='+' + str(twil_data.loc[0]['from_phone_num']),
#     to='+' + str(twil_data.loc[0]['to_phone_num']))
# except :
#   print('Cannot send initialization SMS \n')
#   error = True

# Choose workbook and sheets to pull/update data
sheet = gc.open('games n\' gifts').worksheet('Video Games')
game_pass = gc.open('Game Pass Master List').worksheet('Master List')
giveaways = gc.open('Video Game Giveaways').worksheet('All')

# Create dataframe of all G Sheet records
df = pd.DataFrame(sheet.get_all_records())
gp_df = pd.DataFrame(game_pass.get('A3:Q'), columns = game_pass.get('A2:Q2')[0]).drop_duplicates(subset=['Game'])
g_df = pd.DataFrame(giveaways.get_all_records()).drop_duplicates(subset=['Name'])

# Join data together on name (drop key_0 column as it throws an error) for Game Pass Data and Giveaway data
merged_df = df.merge(gp_df, how='left', left_on=df['Name'], right_on=gp_df['Game'])
merged_df = merged_df.drop(columns=['key_0'])
merged_df = merged_df.merge(g_df, how='left', left_on=merged_df['Name'], right_on=g_df['Name'], suffixes=['_original','_epic'])
merged_df = merged_df.drop(columns=['key_0'])

# Replace empty strings with Null for numeric data
df[['Steam Price', 'Switch Price', 'XBOX Price', 'Min Price',
    'Steam Min Price', 'Switch Min Price', 'XBOX Min Price', 'Deku Min Price',
    'Current Highest Discount %']] = df[['Steam Price', 'Switch Price', 'XBOX Price', 'Min Price',
                                         'Steam Min Price', 'Switch Min Price', 'XBOX Min Price',
                                         'Deku Min Price','Current Highest Discount %']].replace('',float(np.nan))

# Create discount dataframe for Twilio message
# discount_df = pd.DataFrame(columns=['Name', 'System', 'Discount Percent', 'Current Price', 'Link'])

# Blank out all checks
df['Check'] = ''

# Create a session and update headers
sesh = requests.Session()
sesh.headers.update({'User-Agent' : 'Mozilla/5.0'})

# Parse data from DekuDeals.com on historic lowest price and how long to beat if missing
def getDekuData(soup, i) :
  for j in soup.find_all('td', {'class' : 'text-right pl-3'}) :
    if j.text is not None and j.text.strip()[1:] != '' :
      price = float(j.text.strip()[1:].replace('ree','0').replace(',', '.'))
      if price < df.loc[i]['Deku Min Price'] :
        df.at[i, 'Deku Min Price'] = price
  if df.loc[i]['How Long to Beat'] == '' :
    for p in soup.find_all('a') :
      if p.text == 'How Long To Beat' :
        hltb = p.parent.next_sibling.next_sibling.text
        hltb = hltb[hltb.find(':')+2:hltb.find('hour')].strip()
        if '½' in hltb :
          hltb = float(hltb[:hltb.find('½')]) + 0.5
        else :
          hltb = float(hltb)
        df.at[i, 'How Long to Beat'] = hltb
  if soup.find('a', {'class' : 'metacritic'}) is not None :
    if soup.find('a', {'class' : 'metacritic'}).find('span').text != 'tbd' :
      df.at[i, 'Metacritic Rating (Deku)'] = int(soup.find('a', {'class' : 'metacritic'}).find('span').text)

def getHowLong (i, soup) :
  try :
    hltb = soup.find('div', {'class' : 'how-long-to-beat-single'}).find('span', {'class' : 'value'}).text.strip()
    df.at[i, 'How Long to Beat'] = int(hltb[:hltb.find('h')]) + round(int(hltb[hltb.find('h')+1:-1].strip())/60,1)
  except :
    pass

def getggData(i, console, link) :
  try :
    soup = BeautifulSoup(sesh.get(link).content, 'lxml')
    if df.loc[i]['GG Deals ' + console + ' Link'] == '' :
      df.at[i, 'GG Deals ' + console + ' Link'] = link
    if soup.find('span', {'class' : 'expired lowest-recorded price-hl price'}) is not None :
      try :
        df.at[i, console + ' Min Price'] = float(soup.find('span', {'class' : 'expired lowest-recorded price-hl price'}).find('span', {'class' : 'price-inner numeric'}).text.strip()[1:].replace('ree', '0'))
      except :
        df.at[i, console + ' Min Price'] = float(soup.find('span', {'class' : 'expired lowest-recorded price-hl price'}).find('span', {'class' : 'price-inner numeric'}).text.strip()[2:].replace('ee', '0'))
    elif soup.find('a', {'href' : '#official-stores'}) is not None:
      if soup.find('a', {'href' : '#official-stores'}).find('span', {'class' : 'price-inner numeric'}) is not None :
        df.at[i, console + ' Min Price'] = float(soup.find('a', {'href' : '#official-stores'}).find('span', {'class' : 'price-inner numeric'}).text.strip()[1:].replace('ree', '0'))
    elif soup.find('span', {'class' : 'lowest-recorded price-hl price'}) is not None:
      df.at[i, console + ' Min Price'] = float(soup.find('span', {'class' : 'lowest-recorded price-hl price'}).find('span', {'class' : 'price-inner numeric'}).text.strip()[1:].replace('ree', '0'))
    else :
      pass
    getHowLong(i, soup)
  except :
    df.at[i, 'Check'] = 'GG Deals error for ' + console

def replaceChar(name):
    for c in [':','&', '_', ',', '™', '!', '-', '+','\'','.'] :
        name = name.replace(c, '')
    name = name.replace(' ', '-')
    name = name.replace('--', '-')
    name = name.replace('---', '-')
    return name

def getXboxLink(i) :
  e = ['-xbox-pc', '-xbox-one', '-xbox-one-series', '-xbox-series', '-xbox-series-pc', '-xbox-360']
  for suffix in e :
    if requests.get('https://gg.deals/game/' + replaceChar(df.loc[i]['Name'] + suffix)).status_code == 200 :
      getggData(i, 'XBOX', 'https://gg.deals/game/' + replaceChar(df.loc[i]['Name'] + suffix))
      break

# Update the discount data frame if the price of an item has dropped
# def updateDiscountDf(old_price, console, i, discount_df) :
#   if df.loc[i][console + ' Price'] != '' :
#     if old_price > df.loc[i][console + ' Price'] :
#       discount_percent = str(round(((old_price - df.loc[i][console+ ' Price']) / old_price) * 100)) + '%'
#       s = df.loc[i, ['Name', console + ' Price', console + ' Link']].rename({console + ' Price': 'Current Price', console + ' Link': 'Link'})
#       s = pd.concat([s, pd.Series({'System': console, 'Discount Percent' : discount_percent})])
#       discount_df = pd.concat([discount_df, s.to_frame().T], ignore_index=True)

# Iterate through all rows of the dataframe. Update dataframe and G Sheet with values from websites
for i in df.index:
  print(df.loc[i]['Name'])
  # -----------------------------------------------------Steam---------------------------------------------------------#
  # See if there is a steam link and update data
  if df.loc[i]['Steam Link'] != '':
    # Create BS of data from Steam Link
    soup = BeautifulSoup(requests.get(df.loc[i]['Steam Link'] + '?cc=us').content, 'lxml')
    steam_old_price = df.loc[i]['Steam Price']

    # Update name if blank
    if df.loc[i]['Name'] == '':
      df.at[i, 'Name'] = soup.find('span', {'itemprop': 'name'}).text

    # Update the price in the data frame if present
    if soup.find('meta', {'itemprop': 'price'}) is not None :
      df.at[i, 'Steam Price'] = float(soup.find('meta', {'itemprop': 'price'})['content'])
    elif soup.find('div', {'class': 'price bundle_final_price_with_discount'}) is not None :
      df.at[i, 'Steam Price'] = float(soup.find('div', {'class': 'price bundle_final_price_with_discount'}).text[1:])
      steam_msrp = float(soup.find('div', {'class' : 'discount_original_price'}).text[1:])

    #Update min price if there wasn't a min price
    if steam_old_price == '' :
      df.at[i, 'Steam Min Price'] == df.loc[i]['Steam Price']

    # Find Steam discount percentage
    if soup.find('div', {'class' : 'discount_pct'}) :
      df.at[i, 'Steam Discount'] = int(soup.find('div', {'class' : 'discount_pct'}).text[1:-1])

    #updateDiscountDf(steam_old_price, 'Steam', i, discount_df)

    # If there is a sale, find the end date
    try :
      if soup.find('p', {'class': 'game_purchase_discount_countdown'}) is not None:
        sale_end = soup.find('p', {'class': 'game_purchase_discount_countdown'}).text
        if sale_end.find('Offer ends in') > -1:
          sale_end = dt.today()
        else:
          sale_end = dt.strptime(sale_end[sale_end.find('Offer ends') + 11:], '%B %d')
          sale_end = sale_end.replace(year=dt.now().year)
        df.at[i, 'Steam Sale End'] = sale_end.strftime('%m/%d/%Y')
      else:
        df.at[i, 'Steam Sale End'] = ''
        df.at[i, 'Steam Discount'] = ''
    except :
      df.at [i, 'Check'] = 'Steam Sale Date Error'

    # Update Steam review string
    if soup.find('span', {'class': 'game_review_summary_positive', 'itemprop': 'description'}) is not None:
      df.at[i, 'Steam Reviews'] = soup.find('span', {'class': 'game_review_summary_positive',
                                                        'itemprop': 'description'}).text
    elif soup.find('span', {'class': 'game_review_summary positive', 'itemprop': 'description'}) is not None:
      df.at[i, 'Steam Reviews'] = soup.find('span', {'class': 'game_review_summary positive',
                                                        'itemprop': 'description'}).text
    elif soup.find('span', {'class': 'game_review_summary mixed'}) is not None:
      df.at[i, 'Steam Reviews'] = soup.find('span', {'class': 'game_review_summary mixed'}).text

    # Add important tags/traits - Split Screen, Controller Support, Game Type
    if df.loc[i]['Type'] == '' :
      labels = ''
      tags = []
      for j in soup.find_all('div', {'class': 'label'}):
        labels = labels + '/' + j.text
      if 'Shared/Split' in labels:
        df.at[i, 'Steam Local Multiplayer'] = 'x'
      if 'controller support' in labels.lower():
        s = labels[0:labels.lower().find('controller') - 1]
        s = s[s.rfind('/') + 1:]
        df.at[i, 'Steam Controller Support'] = s
      for k in soup.find_all('a', {'class': 'app_tag'}):
        tags.append(k.text.strip() + '/')
      df.at[i, 'Type'] = ''.join(tags[0:4])[:-1]

    # Find # of reviews
    if soup.find('meta', {'itemprop': 'reviewCount'}) is not None:
      df.at[i, 'Steam # of Reviews'] = int(soup.find('meta', {'itemprop': 'reviewCount'})['content'])

    # Find Positive review %
    if soup.find('span', {'class': 'responsive_reviewdesc_short'}) is not None:
      s = soup.find('span', {'class': 'responsive_reviewdesc_short'}).text.strip()[1:]
      if s[:s.find('%')].isnumeric():
        df.at[i, 'Steam % Positive Reviews'] = int(s[:s.find('%')])

    # Find category of Steam Deck Compatibility and update if applicable
    try :
      deck_dict = json.loads(soup.find(id='application_config').get('data-deckcompatibility'))
      if deck_dict['resolved_category'] == 1:
        df.at[i, 'Steam Deck Compatibility'] = 'Unsupported'
        df.at[i, 'Manual Steam Deck Verification'] = 'N'
      elif deck_dict['resolved_category'] == 3:
        df.at[i, 'Steam Deck Compatibility'] = 'Verified'
        df.at[i, 'Manual Steam Deck Verification'] = 'Y'
      elif deck_dict['resolved_category'] == 2:
        df.at[i, 'Steam Deck Compatibility'] = 'Playable'
    except :
      df.at[i, 'Check'] = 'Steam Deck Compatibility Error'

    # Update Release date
    if soup.find('div', {'class' : 'grid_content grid_date'}) is not None and df.loc[i]['Release Date'] == '' and pd.isnull(df.loc[i]['Steam Price']) :
      df.at[i, 'Release Date'] = soup.find('div', {'class' : 'grid_content grid_date'}).text.strip()

    # Update whether there is a Steam Demo
    for b in soup.find_all('a', {'class': 'btnv6_blue_hoverfade btn_medium'}):
      if b.has_attr('href'):
        if 'Download Demo' in b['href']:
          df.at[i, 'Steam Demo'] = 'x'
    for a in soup.find_all('a', {'class': 'btn_green_steamui btn_medium'}):
      if a.has_attr('href'):
        if 'Download' in a['href']:
          df.at[i, 'Steam Demo'] = 'x'

    # Update Steam DB link if there isn't one
    if df.loc[i]['Steam DB Link'] == '' :
      steam_app_id = df.loc[i]['Steam Link'][df.loc[i]['Steam Link'].find('app/')+4:]
      steam_app_id = steam_app_id[:steam_app_id.find('/')]
      df.at[i, 'Steam DB Link'] = 'https://steamdb.info/app/' + steam_app_id

    # Update MetaCritic Score
    if soup.find('div', {'class' : 'score high'}) is not None :
      df.at[i, 'Metacritic Rating (PC)'] = int(soup.find('div', {'class' : 'score high'}).text)

    if df.loc[i]['GG Deals PC Link'] == '' or pd.isnull(df.loc[i]['PC Min Price']) :
      link = 'https://gg.deals/game/' + replaceChar(df.loc[i]['Name'])
      try :
        getggData(i, 'PC', link)
      except :
        df.at[i, 'Check'] = 'GG PC Link'
    else :
      getggData(i, 'PC', df.loc[i]['GG Deals PC Link'])

    if pd.isnull(df.loc[i]['Steam Min Price']) :
      df.at[i, 'Steam Min Price'] = df.loc[i]['Steam Price']

    if df.loc[i]['Switch Link'] == '' :
      if requests.get('https://gg.deals/game/' + replaceChar(df.loc[i]['Name']) + '-nintendo-switch').status_code == 200 :
        df.at[i, 'Check'] = 'Add Switch Link'
  # -----------------------------------------------------Switch---------------------------------------------------------#
  # If there is a Nintendo Link, populate dataframe
  if df.loc[i]['Switch Link'] != '':
      switch_soup = BeautifulSoup(requests.get(df.loc[i]['Switch Link']).content, 'lxml')
      switch_dict = json.loads(switch_soup.find('script', {'id' : '__NEXT_DATA__'}).text)

      # Switch sku needed to determine product to gather data from
      sku = switch_dict['props']['pageProps']['linkedData'][0]['sku']
      key = 'StoreProduct:{\"sku\":\"'+ sku + '\",\"locale\":\"en_US\"}'

      # If name is blank, populate
      if df.loc[i]['Name'] == '':
        df.at[i, 'Name'] = switch_dict['props']['pageProps']['linkedData'][0]['name']

      # Track price info, take old switch price for discount data frame
      switch_old_price = float(df.loc[i]['Switch Price'])
      if switch_dict['props']['pageProps']['initialApolloState'][key]['prices({"personalized":false})']['minimum'] is not None :
        amount_off = float(switch_dict['props']['pageProps']['initialApolloState'][key]['prices({"personalized":false})']['minimum']['amountOff'])
        switch_msrp = float(switch_dict['props']['pageProps']['initialApolloState'][key]['prices({"personalized":false})']['minimum']['regularPrice'])
        switch_final_price = float(switch_dict['props']['pageProps']['initialApolloState'][key]['prices({"personalized":false})']['minimum']['finalPrice'])
        df.at[i, 'Switch Price'] = switch_final_price
        try :
          df.at[i, 'Switch Discount'] = int((amount_off/switch_msrp)*100)
        except :
          pass

      # If there is a sale, find the end date
      sale_end_date = switch_dict['props']['pageProps']['initialApolloState'][key]['eshopDetails({"personalized":false})']
      if sale_end_date is not None :
        if sale_end_date['discountPriceEnd'] is not None :
          sale_end_date = dt.strptime(sale_end_date['discountPriceEnd'],'%Y-%m-%dT%H:%M:%SZ')
          df.at[i, 'Switch Sale End'] = sale_end_date.strftime('%m/%d/%Y')
        else :
          df.at[i, 'Switch Sale End'] = ''
          df.at[i, 'Switch Discount'] = ''

      # If 'Type' has not already been populated from Steam data pull data from Genre section of page
      try :
        if df.loc[i]['Type'] == '':
          df.at[i, 'Type'] = '/'.join(k['label'] for k in switch_dict['props']['pageProps']['initialApolloState'][key]['genres'])
      except :
        pass

      # Find the number of single system players
      if switch_dict['props']['pageProps']['initialApolloState'][key]['playersMaxLocal'] is not None :
        df.at[i, 'Switch Max Single System Players'] = int(switch_dict['props']['pageProps']['initialApolloState'][key]['playersMaxLocal'])
      elif switch_dict['props']['pageProps']['initialApolloState'][key]['playersMaxLocal'] is not None:
        df.at[i, 'Switch Max Single System Players'] = int(switch_dict['props']['pageProps']['initialApolloState'][key]['playersMax'])
      else :
        pass

      # Get lowest price for Switch
      if df.loc[i]['GG Deals Switch Link'] == '' :
        start = df.loc[i]['Switch Link'].find('products/')
        end = df.loc[i]['Switch Link'].find('-switch')
        link = 'https://gg.deals/game/' + df.loc[i]['Switch Link'][start+9:end] + '-nintendo-switch'
        try :
          getggData(i, 'Switch', link)
        except :
          df.at[i, 'Check'] = 'GG Switch Link'
      else :
        getggData(i, 'Switch', df.loc[i]['GG Deals Switch Link'])
      # If a game goes on sale or price drops add details to the discount dataframe
      # updateDiscountDf(switch_old_price, 'Switch', i, discount_df)

      # Mark if Switch is 'Cloud Version'
      # if deku_link.find('cloud-version') > -1 :
      #  df.at[i, 'Switch Cloud Version'] = 'x'
  # -----------------------------------------------------XBOX---------------------------------------------------------#
  # If there is an XBOX Link populate data, do nothing otherwise
  if df.loc[i]['XBOX Link'] != '':
    xbox_soup = BeautifulSoup(requests.get(df.loc[i]['XBOX Link']).content, 'lxml')
    xbox_old_price = float(df.loc[i]['XBOX Price'])

    # Prices from website
    xbox_msrp = xbox_soup.find('span', {'class': 'Price-module__boldText___vmNHu Price-module__moreText___q5KoT'})
    xbox_original_price = xbox_soup.find('span', {'class' : 'Price-module__originalPrice___+jfaT'})
    xbox_final_price = xbox_soup.find('span', {'class': 'Price-module__boldText___vmNHu Price-module__moreText___q5KoT Price-module__listedDiscountPrice___67yG1'})

    # Try these prices if the other ones don't work
    xbox_msrp = xbox_soup.find('span', {'class':'Price-module__boldText___1i2Li Price-module__moreText___sNMVr'})
    xbox_original_price = xbox_soup.find('span', {'class' : 'Price-module__originalPrice___XNCxs'})
    xbox_final_price = xbox_soup.find('span', {'class': 'Price-module__boldText___1i2Li Price-module__moreText___sNMVr Price-module__listedDiscountPrice___A-+d5'})

    # Use discount price if it is not None, otherwise use original price
    try :
      if xbox_final_price is None :
        xbox_final_price = ''
      if xbox_final_price != '' and 'for' not in xbox_final_price.text :
        df.at[i, 'XBOX Price'] = float(xbox_final_price.text[1:].replace('+', ''))
        xbox_original_price = float(xbox_original_price.text[1:].replace('+', ''))
        df.at[i, 'XBOX Discount'] = int(((xbox_original_price - df.loc[i]['XBOX Price'])/xbox_original_price)*100)
      else :
        df.at[i, 'XBOX Price'] = float(xbox_msrp.text[1:].replace('+', '').replace('ree','0'))
        df.at[i, 'XBOX Discount'] = ''
    except :
        df.at[i, 'Check'] = 'XBOX Price'
        error = True

    # If a game goes on sale or price drops add details to the discount dataframe
    # updateDiscountDf(xbox_old_price, 'XBOX', i, discount_df)

    # Update name if blank
    if df.loc[i]['Name'] == '':
      df.at[i, 'Name'] = xbox_soup.find('h1').text

    # Find max amount of players for XBOX locally
    for g in xbox_soup.find_all('li'):
      if g.has_attr('class') and 'local' in g.text:
        df.at[i, 'XBOX Max Single System Players'] = int(g.text[-2])
      elif g.has_attr('class' and 'Single player' in g.text):
        df.at[i, 'XBOX Max Single System Players'] = 1

    # Add text to Type column if empty
    try :
      if df.loc[i]['Type'] == '':
        df.at[i, 'Type'] = xbox_soup.find('div', {'class': 'typography-module__xdsSubTitle1___N02-X ProductDetailsHeader-module__productInfoLine___W-v+p'}).findChildren('span')[0].text.replace(' • ', '/')
    except :
      pass

    # If there is a sale, find the end date
    for b in xbox_soup.find_all('div', {'class': 'typography-module__xdsCaption___RlQY3 ProductTags-module__tagItem___9rKan'}) :
      if 'ends in' in b['title']:
        sale_end = int(b['title'][b['title'].find('ends') + 8:-4].strip())
        sale_end = dt.today() + timedelta(days=sale_end)
        df.at[i, 'XBOX Sale End'] = sale_end.strftime('%m/%d/%Y')

    if df.loc[i]['XBOX Sale End'] != '':
      if dt.today() + timedelta(hours=-7) > dt.strptime(df.loc[i]['XBOX Sale End'], '%m/%d/%Y'):
        df.at[i, 'XBOX Sale End'] = ''
        df.at[i, 'XBOX Discount'] = ''

    # Check if game is included in XBOX Game Pass
    for t in xbox_soup.find_all('h2', {'class' : 'SizeAdaptiveItemSliderChannel-module__channelTitleText___SexCs typography-module__xdsH6___c1AoA'}):
      if t.text == 'Included in these subscriptions' :
        df.at[i, 'XBOX Game Pass Current'] = 'x'
        df.at[i, 'Check'] = 'Game Pass Current'

    # Check GG Deals for best lowest official price on XBOX
    if df.loc[i]['GG Deals XBOX Link'] == '' :
      getXboxLink(i)
    else :
      getggData(i, 'XBOX', df.loc[i]['GG Deals XBOX Link'])
  # -----------------------------------------------------Comparison---------------------------------------------------------#
  # Determine system preference
  if df.loc[i]['Steam Deck Compatibility'] == 'Unsupported':
    df.at[i, 'Manual Steam Deck Verification'] = 'N'
    if df.loc[i]['Switch Cloud Version'] != 'x' and df.loc[i]['Switch Link'] != '':
      df.at[i, 'Pref'] = 'Switch'
    elif df.loc[i]['XBOX Link'] != '':
      df.at[i, 'Pref'] = 'XBOX'
    else:
      df.at[i, 'Pref'] = 'Steam'
  elif df.loc[i]['Steam Deck Compatibility'] == 'Verified' or df.loc[i]['Manual Steam Deck Verification'] == 'Y' :
    if df.loc[i]['Switch Max Single System Players'] != '':
      if df.loc[i]['Switch Max Single System Players'] > 1:
        df.at[i, 'Pref'] = 'Switch'
      elif df.loc[i]['XBOX Max Single System Players'] != '':
        if df.loc[i]['XBOX Max Single System Players'] > 1:
          df.at[i, 'Pref'] = 'XBOX'
      else:
        df.at[i, 'Pref'] = 'Steam'
    elif df.loc[i]['XBOX Max Single System Players'] != '':
      if df.loc[i]['XBOX Max Single System Players'] > 1:
        df.at[i, 'Pref'] = 'XBOX'
      else:
        df.at[i, 'Pref'] = 'Steam'
    else:
      df.at[i, 'Pref'] = 'Steam'
  elif df.loc[i]['Switch Cloud Version'] == 'x' and df.loc[i]['XBOX Link'] != '':
      df.at[i, 'Pref'] = 'XBOX'
  elif df.loc[i]['Switch Link'] != '':
    df.at[i, 'Pref'] = 'Switch'
  elif df.loc[i]['XBOX Link'] != '':
    df.at[i, 'Pref'] = 'XBOX'
  else:
    df.at[i, 'Pref'] = 'Steam'

  # Find the minimum price from all systems and set the field for system with current lowest price
  df.at[i, 'Current Lowest Price'] = df.loc[i, 'Steam Price' : 'XBOX Price'].min()
  plat = pd.to_numeric(df.loc[i, 'Steam Price' : 'XBOX Price']).idxmin()
  if df.loc[i]['Current Lowest Price'] == df.loc[i][df.loc[i]['Pref'] + ' Price']:
    df.at[i, 'Current Lowest Price System'] = df.loc[i]['Pref']
  elif isinstance(plat, str) :
    df.at[i, 'Current Lowest Price System'] = plat[:plat.find(' Price')]

  # Find the highest discount % and day that sale ends
  if df.loc[i]['Steam Sale End'] != '' or df.loc[i]['Switch Sale End'] != '' or df.loc[i]['XBOX Sale End'] != '':
    df.at[i, 'Current Highest Discount %'] = df.loc[i][df.loc[i]['Current Lowest Price System'] + ' Discount']
    df.at[i, 'Current Lowest Price Sale End'] = df.loc[i][df.loc[i]['Current Lowest Price System'] + ' Sale End']
  else:
    df.at[i, 'Current Lowest Price Sale End'] = ''
    df.at[i, 'Current Highest Discount %'] = np.nan

  # Searches Deku Deals for minimum price for XBOX and Switch
  # if pd.isnull(df.loc[i]['Deku Min Price']) and (df.loc[i]['Switch Link'] != '' or df.loc[i]['XBOX Link'] != ''):
  #   if df.loc[i]['Deku Link'] == '' :
  #     if df.loc[i]['Switch Link'] != '' :
  #       start = df.loc[i]['Switch Link'].find('products/')
  #       deku_link = 'https://www.dekudeals.com/items/' + df.loc[i]['Switch Link'][start+9:-8]
  #       deku_link2 = 'https://www.dekudeals.com/items/' + df.loc[i]['Switch Link'][start+9]
  #     if df.loc[i]['XBOX Link'] != '' :
  #       deku_link3 = 'https://www.dekudeals.com/items/' + df.loc[i]['XBOX Link'][39:df.loc[i]['XBOX Link'].rfind('/')]
  #     if df.loc[i]['Switch Link'] == '' :
  #       deku_link = deku_link3
  #   else :
  #     deku_link = df.loc[i]['Deku Link']
  #     deku_link2 = deku_link
  #     deku_link3 = deku_link
  #   deku_soup = BeautifulSoup(requests.get(deku_link).content,'lxml')
  #   if deku_soup.find('main', {'class' : 'container-fluidish mb-4 search-container'}) is not None :
  #     deku_soup = BeautifulSoup(requests.get(deku_link2).content,'lxml')
  #     if deku_soup.find('main', {'class' : 'container-fluidish mb-4 search-container'}) is not None :
  #       deku_soup = BeautifulSoup(requests.get(deku_link3).content,'lxml')
  #       getDekuData(deku_soup, i)
  #       df.at[i,'Deku Link'] = deku_link3
  #     else :
  #       getDekuData(deku_soup, i)
  #       df.at[i, 'Deku Link'] = deku_link2
  #   else :
  #     getDekuData(deku_soup, i)
  #     df.at[i, 'Deku Link'] = deku_link

  if df.loc[i]['Min Price Platform'] == '' :
    min = pd.to_numeric(df.loc[i, 'Steam Min Price' : 'XBOX Min Price']).idxmin()
    if isinstance(min, str) :
      df.at[i, 'Min Price Platform'] = min[:min.find(' Min')]

  # Update if game has been removed from Game Pass, given away from Epic, Amazon or GOG
  if merged_df.loc[i]['Status'] == 'Active' :
    df.at[i, 'XBOX Game Pass Current'] = 'x'
    df.at[i, 'Check'] = 'Game Pass Current'
    df.at[i, 'Min Price Platform'] = 'XBOX Game Pass'
  elif pd.notnull(merged_df.loc[i]['Removed']) :
    df.at[i, 'XBOX Game Pass Removal Date'] = merged_df.loc[i]['Removed']
    df.at[i, 'Check'] = 'Game Pass Removed'
    df.at[i, 'XBOX Game Pass Current'] = ''
    df.at[i, 'Min Price Platform'] = 'XBOX Game Pass'
  if pd.notnull(merged_df.loc[i]['Name_epic']) :
    df.at[i, 'Giveaway'] = 'x'
    df.at[i, 'Check'] = merged_df.loc[i]['Where'] + ' Giveaway'
    df.at[i, 'Min Price Platform'] = merged_df.loc[i]['Where'] + ' Giveaway'
    if merged_df.loc[i]['Date'] != '' :
      df.at[i, 'Giveaway Date'] = merged_df.loc[i]['Date']
    else :
      df.at[i, 'Giveaway Date'] = merged_df.loc[i]['Year']
    if merged_df.loc[i]['Metacritic PC Score'] not in ('N/A', '') and df.loc[i]['Metacritic Rating (PC)'] == '' :
      df.at[i, 'Metacritic Rating (PC)'] = merged_df.loc[i]['Metacritic PC Score']

  # Update lowest price on record if it's lower than the current one
  df.at[i, 'Min Price'] = df.loc[i, 'Steam Min Price' : 'XBOX Min Price'].min()
  if df.loc[i]['XBOX Game Pass Current'] == 'x' or df.loc[i]['Giveaway'] == 'x' or df.loc[i]['XBOX Game Pass Removal Date'] != '':
    df.at[i, 'Min Price'] = 0
  if df.loc[i]['Current Lowest Price'] != '' :
    if df.loc[i]['Current Lowest Price'] < df.loc[i]['Min Price'] :
      df.at[i, 'Min Price'] = df.loc[i]['Current Lowest Price']
      df.at[i, 'Min Price Platform'] = df.loc[i]['Current Lowest Price System']
      df.at[i, df.loc[i]['Current Lowest Price System'] + ' Min Price'] = df.loc[i]['Current Lowest Price']

  if pd.isnull(df.loc[i]['Current Highest Discount %']) or df.loc[i]['Current Highest Discount %'] == '' :
    df.at[i, 'Current Highest Discount %'] = 0

  if df.loc[i]['How Long to Beat'] != '' :
    df.at[i, 'Dollar per Hour'] = round(float(df.loc[i]['Current Lowest Price'] / df.loc[i]['How Long to Beat']), 2)

  if df.loc[i]['Dollar per Hour'] != '' :
    if df.loc[i]['Dollar per Hour'] <= 1 :
      df.at[i, 'DPH Worth'] = 'Y'
    else :
      df.at[i, 'DPH Worth'] = 'N'

  # If the lowest price is at it's all time low, the system is correct then check to see if you should buy it
  if df.loc[i]['Min Price Platform'] in ('XBOX Game Pass', 'Epic Giveaway', 'Amazon Giveaway', 'GOG Giveaway'):
    pass
  elif df.loc[i]['Pref'] == 'Steam' and df.loc[i]['Current Lowest Price'] == df.loc[i]['Min Price'] and (df.loc[i]['Current Highest Discount %'] >= 50 or df.loc[i]['DPH Worth'] == 'Y') :
    if df.loc[i]['Current Lowest Price System'] != 'Steam' :
      if df.loc[i]['Current Lowest Price System'] == 'XBOX' :
        df.at[i, 'Check'] = 'Buy ' + df.loc[i]['Current Lowest Price System'] + ' (not pref)'
      elif df.loc[i]['Switch Cloud Version'] != 'x' :
        df.at[i, 'Check'] = 'Buy ' + df.loc[i]['Current Lowest Price System'] + ' (not pref)'
    else :
      df.at[i, 'Check'] = 'Buy Steam'
  elif df.loc[i]['Pref'] != 'Steam' and df.loc[i]['Current Lowest Price'] == df.loc[i]['Min Price'] and (df.loc[i]['Current Highest Discount %'] >= 50 or df.loc[i]['DPH Worth'] == 'Y') :
    if df.loc[i]['Current Lowest Price System'] == 'Steam' and df.loc[i]['Manual Steam Deck Verification'] == 'Y':
      df.at[i, 'Check'] = 'Buy Steam (not pref)'
    elif df.loc[i]['Current Lowest Price System'] == 'Steam' and df.loc[i]['Manual Steam Deck Verification'] not in ['Y','N']:
      df.at[i, 'Check'] = 'Steam Deck Compatibility'
    elif df.loc[i]['Switch Cloud Version'] == 'x' and df.loc[i]['Current Lowest Price System'] == 'Switch':
      pass
    elif df.loc[i]['Current Lowest Price System'] == 'Steam' and df.loc[i]['Manual Steam Deck Verification'] == 'N':
      pass
    else :
      df.at[i, 'Check'] = df.at[i, 'Check'] = 'Buy ' + df.loc[i]['Current Lowest Price System']


# Fill nulls with blanks and convert Priority to string because G Sheets/Gspread can't process numpy data types
df = df.fillna('')
df = df.astype({'Priority': 'str'})

# Create a list of lists of the data frame so the sheet can update
arr = []
arr.append(list(df.columns.array))
for i in df.index:
  arr.append(list(df.iloc[i].array))

# Updates entire sheet with list of lists arr
sheet.update('A1', arr)

# Send SMS message on discounts and whether there was an error
# try :
#   if discount_df.empty == False:
#     body = "Here is the list of discounted video games from the most recent script run: " + "\n" * 2 + "\n".join(" - ".join(map(str, xs)) for xs in discount_df.itertuples(index=False))
#     message = client.messages.create(
#       body=body,
#       from_='+' + str(twil_data.loc[0]['from_phone_num']),
#       to='+' + str(twil_data.loc[0]['to_phone_num']))

#   if error is True:
#     message = client.messages \
#       .create(
#       body="The video game script has run with an error.",
#       from_='+' + str(twil_data.loc[0]['from_phone_num']),
#       to='+' + str(twil_data.loc[0]['to_phone_num']))
#   else:
#     message = client.messages \
#       .create(
#       body="The video game script run has finished with no errors.",
#       from_='+' + str(twil_data.loc[0]['from_phone_num']),
#       to='+' + str(twil_data.loc[0]['to_phone_num']),)
# except :
#   print('\nCannot send script log SMS')

<ipython-input-7-6584a1e8cf52>:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  'Deku Min Price','Current Highest Discount %']].replace('',float(np.nan))


198X
33 Immortals


<ipython-input-7-6584a1e8cf52>:440: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  plat = pd.to_numeric(df.loc[i, 'Steam Price' : 'XBOX Price']).idxmin()
<ipython-input-7-6584a1e8cf52>:484: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min = pd.to_numeric(df.loc[i, 'Steam Min Price' : 'XBOX Min Price']).idxmin()


80 Days
9 Monkeys of Shaolin + Ash of Gods + Redeemer: Bundle
A Frog's Tale


<ipython-input-7-6584a1e8cf52>:440: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  plat = pd.to_numeric(df.loc[i, 'Steam Price' : 'XBOX Price']).idxmin()
<ipython-input-7-6584a1e8cf52>:484: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min = pd.to_numeric(df.loc[i, 'Steam Min Price' : 'XBOX Min Price']).idxmin()


...

Inferius


<ipython-input-7-6584a1e8cf52>:440: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  plat = pd.to_numeric(df.loc[i, 'Steam Price' : 'XBOX Price']).idxmin()
<ipython-input-7-6584a1e8cf52>:484: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min = pd.to_numeric(df.loc[i, 'Steam Min Price' : 'XBOX Min Price']).idxmin()


Preserve
Two Point Museum
What Never Was: Chapter II


<ipython-input-7-6584a1e8cf52>:440: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  plat = pd.to_numeric(df.loc[i, 'Steam Price' : 'XBOX Price']).idxmin()
<ipython-input-7-6584a1e8cf52>:484: FutureWarning: The behavior of Series.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  min = pd.to_numeric(df.loc[i, 'Steam Min Price' : 'XBOX Min Price']).idxmin()
<ipython-input-7-6584a1e8cf52>:556: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna('')
<ipython-input-7-6584a1e8cf52>:566: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and r

{'spreadsheetId': '1Wt1IuEkaQY-0ajwGR2SemfKuVJ4HO5D41rJHUXm-lQY',
 'updatedRange': "'Video Games'!A1:BE637",
 'updatedRows': 637,
 'updatedColumns': 57,
 'updatedCells': 36309}

404

<ipython-input-21-c7a2451658b7>:12: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', arr)


{'spreadsheetId': '1Wt1IuEkaQY-0ajwGR2SemfKuVJ4HO5D41rJHUXm-lQY',
 'updatedRange': "'Video Games'!A1:BE634",
 'updatedRows': 634,
 'updatedColumns': 57,
 'updatedCells': 36138}